In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
import sympy as sp
import time

In [2]:
# Definition of the potential

# Parameters of the potential
a = 0.02
x_min = 10 # Position of the second minimum, with U(x_min)=(a*x_min**3*c**2)/(x_min+2*c),
          # the first minimum is in x=0, with f(0)=0  
c = 5

def potential (x, a=a, x_min=x_min, c=c):
    U = a*(x-x_min)**2*(x-c)*(x+(x_min*c)/(x_min+2*c)) + (a*x_min**3*c**2)/(x_min+2*c)
    return U

In [3]:
# Definition of the pulling force seen as the gradient of an armonic potential of the form: 
# F = (k/2)*(x-x_0)**2, where 'k' is the analogous of the elastic constant, and x_0 is the position of the minimum

k=10
x_center=0
def external_force(x, x_center, k=k):
    return -k*(x-x_center)


In [4]:
# Parameters
n_tries = 150
x_center = 0
total_time = 100
dt = 0.005
v_external_force = x_min/total_time
n_steps= round(total_time/dt)
gamma= 1 # friction coefficient 
kbT=1
D = kbT/gamma

std = 1/np.sqrt(2*((a*x_min)/(x_min+2*c)*(3*c**2+x_min**2+2*x_min*c)+k/2)/kbT) 


In [5]:
# Definition of the initial probability function (Boltzmann distribution)

def eq_distribution_in(x):
    p = np.exp((-potential(x)-(k/2)*(x-x_center)**2)/kbT)
    return p

# Second order approximation (gaussian)
def approx_in(x):
    p = np.exp((-((a*x_min)/(x_min+2*c)*(3*c**2+x_min**2+2*x_min*c)+k/2)*x**2)/kbT)
    return p
std = 1/np.sqrt(2*((a*x_min)/(x_min+2*c)*(3*c**2+x_min**2+2*x_min*c)+k/2)/kbT) # standard deviation of the initial distribution


In [6]:
# Definition of the final probability function (Boltzmann distribution)

def eq_distribution_fin(x):
    p = np.exp((-potential(x)-(k/2)*(x-x_min)**2)/kbT)
    return p

# Second order approximation (gaussian)
def approx_fin(x):
    p = np.exp((-(a*(x_min-c)*(x_min+ (x_min*c)/(x_min+2*c)) + k/2)*(x-x_min)**2 - (a*x_min**3*c**2)/(x_min+2*c))/kbT)
    return p
std_fin = 1/np.sqrt(2*((a*x_min)/(x_min+2*c)*(3*c**2+x_min**2+2*x_min*c)+k/2)/kbT)  # standard deviation of the final distribution


In [7]:
# Plot

fig, ax = plt.subplots()

x = np.linspace(-0.2*x_min, 1.2*x_min, 500)  
x_force = np.linspace(x_center-2, x_center+2, 100)
x_final = np.linspace(x_min-2, x_min+2, 100)

line, = ax.plot(x_force, k/2*(x_force-x_center)**2)

integral_in, _ = integrate.quad(eq_distribution_in , -5, 5)
int_approx_in, _ = integrate.quad(approx_in , -5, 5)
free_energy_in = - kbT * np.log(int_approx_in)

integral_fin, _ = integrate.quad(eq_distribution_fin , -5+x_min, 5+x_min)
int_approx_fin, _ = integrate.quad(approx_fin , -5+ x_min, 5+ x_min)
free_energy_fin = - kbT * np.log(integral_fin)

plt.plot(x, potential(x), label='potential')
plt.plot(x_force,  5*eq_distribution_in(x_force)/integral_in, label=' initial distribution') # it's multiplied to be able to see it
#plt.plot(x_force, 5*approx_in(x_force)/int_approx_in, label='second order approximation initial')
plt.plot(x_force,  5*approx_in(x_force)/np.max(approx_in(x_force)), label='second order approximation initial')
plt.plot(x_force,k/2*(x_force-x_center)**2, label='external force')
plt.plot(x_final,  5*eq_distribution_fin(x_final)/integral_fin, label=' final distribution') # it's multiplied to be able to see it
#plt.plot(x_final,  5*approx_fin(x_final)/int_approx_fin, label='second order approximation final')
plt.plot(x_final,  5*approx_fin(x_final)/np.max(approx_fin(x_final)), label='second order approximation final')

plt.legend()

plt.show()
print(free_energy_in, free_energy_fin)



In [8]:
# Variation of free energy
d_free_energy = free_energy_fin - free_energy_in
print(d_free_energy)

In [9]:
# Total F_ext (calculated with symbolic derivative):
x_ = sp.symbols('x_')
pot = a*(x_-x_min)**2*(x_-c)*(x_+(x_min*c)/(x_min+2*c)) + (a*x_min**3*c**2)/(x_min+2*c)
# Symbolic derivative
derivative = sp.diff(pot, x_)

def F_tot(x,x_center):
    F = external_force(x, x_center) - derivative.subs(x_,x)
    return F

In [10]:
# Define the iteration process 
def iteration (x_in, x_center, v_external_force=v_external_force, gamma=gamma, kbT=kbT, dt=dt):
    rand = np.random.normal()
    
    # Explicit Euler
    #x_new = x_in + F_tot(x_in, x_center)/gamma * dt + np.sqrt(2*D*dt)*rand

    # Runge-kutta               
    k1x = F_tot(x_in, x_center) / gamma * dt + np.sqrt(2 * D * dt) * rand
    k2x = F_tot(x_in + k1x / 2, x_center + v_external_force*dt/2) / gamma * dt + np.sqrt(2 * D * dt) * rand
    k3x = F_tot(x_in + k2x / 2, x_center + v_external_force*dt/2) / gamma * dt + np.sqrt(2 * D * dt) * rand
    k4x = F_tot(x_in + k3x, x_center) / gamma * dt + np.sqrt(2 * D * dt) * rand 

    x_new = x_in + (k1x + 2 * k2x + 2 * k3x + k4x) / 6 
    
   
    return x_new


In [11]:
# Calculation of trajectory

# Inizialization
w = np.zeros(n_tries) # collects the work done
x_in = np.zeros(n_tries) # collects teh initial positions
x_fin = np.zeros(n_tries) # collects the final positions
x_center=0
x_sim = np.zeros(n_steps+1)

# Total F_ext (calculated with symbolic derivative):
x_ = sp.symbols('x_')
pot = a*(x_-x_min)**2*(x_-c)*(x_+(x_min*c)/(x_min+2*c)) + (a*x_min**3*c**2)/(x_min+2*c)
# Symbolic derivative
derivative = sp.diff(pot, x_)
# Integration
start_time = time.time()
for j in range(n_tries):
    
    x_sim[0] = np.random.normal(0,std)
    x_center = 0
    
    for i in range(1, n_steps+1):
        
        x_sim[i] = iteration(x_sim[i-1], x_center)
                                                         
        w[j] += external_force((x_sim[i-1]+x_sim[i])/2,x_center +v_external_force * dt/2)*(x_sim[i]-x_sim[i-1])

        x_center += v_external_force*dt
        

 
    x_fin[j]=x_sim[-1]
    x_in[j]=x_sim[0]
    
end_time = time.time()
execution_time = end_time - start_time


print(f"Tempo totale di esecuzione: {execution_time} secondi")


In [12]:
# Checks if for any reason some of the works are not_a_number
invalid_indexes=np.where(np.isnan(w))
w = np.delete(w,invalid_indexes)
x_fin = np.delete(x_fin,invalid_indexes)
x_in = np.delete (x_in,invalid_indexes)
print(len(invalid_indexes[0]), "tries have been deleted")

In [13]:
# Computing the exponential of the work
exp_w = np.exp(- w / kbT) 

In [14]:
# Visualizing the dynamics of the last try
tim_ = np.linspace(0,total_time , x_sim.size)
fig, axs = plt.subplots(1, 2, figsize=(10, 5))


axs[0].plot(tim_, x_sim, 'g')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Position')
axs[0].set_title(' Langevin Simulation')

x_plot=x_sim[::int(len(x_sim)/50)] # plots only 50 points
axs[1].plot(x_plot,potential(x_plot), 'ro', label='Simulation')
axs[1].plot(x, potential(x), label='Potential')
axs[1].set_xlabel('Position')
axs[1].set_ylabel('Potential')
axs[1].set_title(' Langevin Simulation')

In [15]:
# Visualizing the distribution of work and its exponential
fig, axs = plt.subplots(1, 2, figsize=(10, 5))


axs[0].hist(w, bins=round(np.sqrt(w.size)), alpha=0.4, color='blue', label='work', density=True)
axs[0].set_xlabel('work')
axs[0].set_ylabel('relative frequency')
axs[0].set_title(' distribution of work')


axs[1].hist(exp_w, bins=round(np.sqrt(w.size)), alpha=0.4, color='red', label='exponential of work', density=False)
axs[1].set_xlabel('exponential of work')
axs[1].set_ylabel('relative frequency')
axs[1].set_title(' distribution of exponential of work')

plt.show()

In [16]:
# Printing the results
print('The first factor of the Jarzynski Relation is: ', np.mean(exp_w))
print('The second factor of the Jarzynski Relation is: ', np.exp(-d_free_energy/kbT))
print('The difference between the two terms is: ', np.abs(np.mean(exp_w)-np.exp(-d_free_energy/kbT)))

In [18]:
# Saving the data
def write_vectors_to_file(file_path, vectors):
    with open(file_path, 'w') as file:
        file.write('x_in'+'\t'+' x_fin'+' \t'+' w' + '\n')
        for i in range(len(vectors[0])):
            column_values = [vector[i] for vector in vectors]

            column_str = ' '.join(map(str, column_values))
            
            file.write(column_str + '\n')


vectors = [x_in, x_fin, w]

file_path = 'results_integration.txt'
write_vectors_to_file(file_path, vectors)